In [1]:
import pandas as pd
import numpy as np

In [2]:
from stat_encoder import *

In [3]:
from scorer_RMSE import RMSE

In [4]:
from xgboost import XGBRegressor

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# загрузка частей train
train = pd.read_csv('train.csv',sep =',')
train2w = pd.read_csv('two_way_inter_train.csv',sep =',') 
train3w = pd.read_csv('three_way_inter_train.csv',sep =',')
train4w = pd.read_csv('four_way_inter_train.csv',sep =',')

Y = np.log( train['price'])
train['area'] = np.log( train['area']  )
train['security_deposit'] = np.log( train['security_deposit']  )
train.drop(['price','Property_ID','locality'] , axis=1 , inplace = True)

In [8]:
# загрузка частей test
test = pd.read_csv('test.csv',sep =',')
test2w = pd.read_csv('two_way_inter_test.csv',sep =',') 
test3w = pd.read_csv('three_way_inter_test.csv',sep =',')
test4w = pd.read_csv('four_way_inter_test.csv',sep =',')

test['area'] = np.log( test['area']  )
test['security_deposit'] = np.log( test['security_deposit']  )
test.drop(['price','Property_ID','locality'] , axis=1 , inplace = True)

In [9]:
train2w.columns = test2w.columns 

In [10]:
train3w.columns = test3w.columns 

In [11]:
# соединение в один датафрейм для train и test

In [12]:
full_train = pd.concat([train , train2w , train3w ,train4w] , axis=1)


In [13]:
full_test = pd.concat([test , test2w , test3w ,test4w] , axis=1)

In [14]:
#Удаление датафреймов из памяти
del test , test2w ,test3w  ,test4w 

In [15]:
del train , train2w ,train3w  ,train4w

In [16]:
del full_test 

Разделение на train , sub_validation и validation в отношении 60% , 20% , 20% соответственно

На train и sub_validation будет производиться отбор признаков и оптимизация гиперпараметров.

Затем модель обучится на (train + sub_validation ) и будет оценена на validation

Для выработки прогноза на test для посылки на сайт модель тренируется на  train + sub_validation + validation 

In [17]:
x_train , x_val , y_train , y_val = train_test_split(full_train , Y , random_state = 67 , test_size = 0.2 )

In [18]:
x_train , x_s_val , y_train , y_s_val = train_test_split(x_train , y_train , random_state = 67 , test_size = 0.25 )

In [19]:
print('Размеры выборок \n'
    ,'train ',x_train.shape , '\n' ,
      'sub_validation ',x_s_val.shape , '\n',
      'validation ',x_val.shape ,sep='')

Размеры выборок 
train (80809, 174)
sub_validation (26937, 174)
validation (26937, 174)


Отбор признаков и статистических показателей для кодирования

Нужно отобрать категориальные признаки, дающие высокий резуьтат по метрике RMSE.

К каждому категориальному признаку применяется кодирование по статистическому показателю.

Оно состоит в следующем:
1. Значения целевой переменной группируется по уровням категории (2 уровня - если признак бинарный)
2. По группам считается стат. показатель (среднее, процентили и т. п.)
3. Значения категориального признака заменяются вычисленными показателями.

Эта техника называется mean encoding.

Нужно также отобрать выгодную комбинацию стат. показателей.

In [20]:
# списов всех категориальных переменных
categorical_features =  x_train.columns[(x_train.columns !='area' ) & (x_train.columns !='security_deposit' )]  

In [21]:
len( categorical_features )

172

In [22]:
# варианты статистических показателей для кодирования.
# q - значит квантиль
stat_parameter = [
    ['mean'] ,
    ['mean', 'q50' ] ,
    ['mean'  ,'q25','q50', 'q75' , 'min','max'] ,
    ['mean'  , 'q50' , 'min','max'] ,

    ['mean' ,'q10', 'q20', 'q30', 'q40', 'q50', 'q60', 'q70', 'q80', 'q90','q25','q50', 'q75' , 'min','max'] ,
 
]

Baseline-модель - Случайный лес с парамерами по умолчанию (100 деревьев и др. параметры)

Используются все признаки и все показатели

In [23]:
# сначала производится кодирование признаков, затем обучение модели
pipe_model_baseline = Pipeline( steps = [
    ('preproc' ,stat_encoder2(  cat_features =categorical_features  , stat_enc =stat_parameter[ 2],verbose = True )) ,
    ('model' , XGBRegressor(n_jobs =3 , verbose = 2, random_state = 45)) ])

In [24]:
# обучение
pipe_model_baseline.fit(x_train , y_train)

Preparing feature (1/172): room
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (2/172): layout_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (3/172): property_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (5/172): bathroom
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (6/172): city
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (8/172): floor
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (10/172): power_backup
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (12/172): air_conditioner
Вычислен

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (86/172): washing_machine x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (87/172): washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (88/172): washing_machine x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (89/172): air_conditioner x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (90/172): air_conditioner x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (91/172): air_conditioner x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (92/172): geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (93/172): CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (94/172): room x layout_type x furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (157/172): layout_type x parking_spaces x geyser/solar x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (158/172): layout_type x power_backup x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (159/172): layout_type x washing_machine x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (160/172): layout_type x air_conditioner x geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (161/172): layout_type x air_conditioner x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (162/172): property_type x parking_spaces x pet_friendly x washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (163/172): property_type x pet_friendly x washing_machine x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (139/172): bathroom x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (140/172): bathroom x air_conditioner x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (141/172): city x floor x air_conditioner
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (142/172): parking_spaces x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (143/172): parking_spaces x air_conditioner x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (144/172): parking_spaces x CCTV/security x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (145/172): washing_machine x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (146/172): air_conditioner x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing featu

Pipeline(steps=[('preproc',
                 stat_encoder2(cat_features=Index(['room', 'layout_type', 'property_type', 'furnish_type', 'bathroom',
       'city', 'parking_spaces', 'floor', 'pet_friendly', 'power_backup',
       ...
       'property_type x pet_friendly x washing_machine x geyser/solar',
       'furnish_type x city x parking_spaces x pet_friendly',
       'furnish_type x city x parking_spaces x lift',
       'furnish_type x c...
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=3, num_parallel_tree=1, random_state=45,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exa

In [25]:
#оценка модели на наборе sub_validation
RMSE( pipe_model_baseline ,  x_s_val ,   y_s_val)

Preparing feature (1/172): room
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (2/172): layout_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (3/172): property_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (5/172): bathroom
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (6/172): city
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (8/172): floor
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (10/172): power_backup
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (12/172): air_conditioner
Вычислен

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (86/172): washing_machine x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (87/172): washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (88/172): washing_machine x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (89/172): air_conditioner x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (90/172): air_conditioner x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (91/172): air_conditioner x neighbourhood
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (92/172): geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (93/172): CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (94/172): room x layout_type x furnish_type
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (9

Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (157/172): layout_type x parking_spaces x geyser/solar x CCTV/security
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (158/172): layout_type x power_backup x washing_machine x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (159/172): layout_type x washing_machine x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (160/172): layout_type x air_conditioner x geyser/solar x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (161/172): layout_type x air_conditioner x CCTV/security x lift
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (162/172): property_type x parking_spaces x pet_friendly x washing_machine
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature (163/172): property_type x pet_friendly x washing_machine x geyser/solar
Вычислены статистики: mean q25 q50 q75 min max 
Preparing feature

-28343.774725597134

RMSE = 28343.77 на всех признаках с кодировкой  'mean'  ,'q25','q50', 'q75' , 'min','max' .


Выводы:
Нужно использовать взаимодействия признаков.
И набор статистических показателей:

In [26]:
# сначала производится кодирование признаков, затем обучение модели
pipe_model_baseline2 = Pipeline( steps = [
    ('preproc' ,stat_encoder2(  cat_features =categorical_features  , stat_enc =stat_parameter[ 0],verbose = True )) ,
    ('model' , XGBRegressor(n_jobs =4 , verbose = 2, random_state = 45)) ])

In [27]:
# обучение
pipe_model_baseline2.fit(x_train , y_train)

Preparing feature (1/172): room
Вычислены статистики: mean 
Preparing feature (2/172): layout_type
Вычислены статистики: mean 
Preparing feature (3/172): property_type
Вычислены статистики: mean 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean 
Preparing feature (5/172): bathroom
Вычислены статистики: mean 
Preparing feature (6/172): city
Вычислены статистики: mean 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean 
Preparing feature (8/172): floor
Вычислены статистики: mean 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean 
Preparing feature (10/172): power_backup
Вычислены статистики: mean 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean 
Preparing feature (12/172): air_conditioner
Вычислены статистики: mean 
Preparing feature (13/172): geyser/solar
Вычислены статистики: mean 
Preparing feature (14/172): CCTV/security
Вычислены статистики: mean 
Preparing feature (15/172): lift
Вычислены статистики: mean 


Вычислены статистики: mean 
Preparing feature (105/172): layout_type x property_type x lift
Вычислены статистики: mean 
Preparing feature (106/172): layout_type x furnish_type x city
Вычислены статистики: mean 
Preparing feature (107/172): layout_type x furnish_type x power_backup
Вычислены статистики: mean 
Preparing feature (108/172): layout_type x bathroom x power_backup
Вычислены статистики: mean 
Preparing feature (109/172): layout_type x city x power_backup
Вычислены статистики: mean 
Preparing feature (110/172): layout_type x city x washing_machine
Вычислены статистики: mean 
Preparing feature (111/172): layout_type x city x air_conditioner
Вычислены статистики: mean 
Preparing feature (112/172): layout_type x parking_spaces x power_backup
Вычислены статистики: mean 
Preparing feature (113/172): layout_type x floor x pet_friendly
Вычислены статистики: mean 
Preparing feature (114/172): layout_type x floor x power_backup
Вычислены статистики: mean 
Preparing feature (115/172): la

Preparing feature (20/172): room x bathroom
Вычислены статистики: mean 
Preparing feature (21/172): room x city
Вычислены статистики: mean 
Preparing feature (22/172): room x parking_spaces
Вычислены статистики: mean 
Preparing feature (23/172): room x floor
Вычислены статистики: mean 
Preparing feature (24/172): room x power_backup
Вычислены статистики: mean 
Preparing feature (25/172): room x washing_machine
Вычислены статистики: mean 
Preparing feature (26/172): room x air_conditioner
Вычислены статистики: mean 
Preparing feature (27/172): room x geyser/solar
Вычислены статистики: mean 
Preparing feature (28/172): room x lift
Вычислены статистики: mean 
Preparing feature (29/172): room x neighbourhood
Вычислены статистики: mean 
Preparing feature (30/172): layout_type x property_type
Вычислены статистики: mean 
Preparing feature (31/172): layout_type x furnish_type
Вычислены статистики: mean 
Preparing feature (32/172): layout_type x bathroom
Вычислены статистики: mean 
Preparing fe

Вычислены статистики: mean 
Preparing feature (119/172): property_type x furnish_type x parking_spaces
Вычислены статистики: mean 
Preparing feature (120/172): property_type x parking_spaces x pet_friendly
Вычислены статистики: mean 
Preparing feature (121/172): property_type x parking_spaces x washing_machine
Вычислены статистики: mean 
Preparing feature (122/172): property_type x parking_spaces x air_conditioner
Вычислены статистики: mean 
Preparing feature (123/172): property_type x air_conditioner x geyser/solar
Вычислены статистики: mean 
Preparing feature (124/172): property_type x geyser/solar x CCTV/security
Вычислены статистики: mean 
Preparing feature (125/172): property_type x geyser/solar x lift
Вычислены статистики: mean 
Preparing feature (126/172): furnish_type x bathroom x geyser/solar
Вычислены статистики: mean 
Preparing feature (127/172): furnish_type x bathroom x lift
Вычислены статистики: mean 
Preparing feature (128/172): furnish_type x city x floor
Вычислены стат

Pipeline(steps=[('preproc',
                 stat_encoder2(cat_features=Index(['room', 'layout_type', 'property_type', 'furnish_type', 'bathroom',
       'city', 'parking_spaces', 'floor', 'pet_friendly', 'power_backup',
       ...
       'property_type x pet_friendly x washing_machine x geyser/solar',
       'furnish_type x city x parking_spaces x pet_friendly',
       'furnish_type x city x parking_spaces x lift',
       'furnish_type x c...
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=4, num_parallel_tree=1, random_state=45,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exa

In [28]:
#оценка модели на наборе sub_validation
RMSE( pipe_model_baseline2 ,  x_s_val ,   y_s_val)

Preparing feature (1/172): room
Вычислены статистики: mean 
Preparing feature (2/172): layout_type
Вычислены статистики: mean 
Preparing feature (3/172): property_type
Вычислены статистики: mean 
Preparing feature (4/172): furnish_type
Вычислены статистики: mean 
Preparing feature (5/172): bathroom
Вычислены статистики: mean 
Preparing feature (6/172): city
Вычислены статистики: mean 
Preparing feature (7/172): parking_spaces
Вычислены статистики: mean 
Preparing feature (8/172): floor
Вычислены статистики: mean 
Preparing feature (9/172): pet_friendly
Вычислены статистики: mean 
Preparing feature (10/172): power_backup
Вычислены статистики: mean 
Preparing feature (11/172): washing_machine
Вычислены статистики: mean 
Preparing feature (12/172): air_conditioner
Вычислены статистики: mean 
Preparing feature (13/172): geyser/solar
Вычислены статистики: mean 
Preparing feature (14/172): CCTV/security
Вычислены статистики: mean 
Preparing feature (15/172): lift
Вычислены статистики: mean 


Вычислены статистики: mean 
Preparing feature (109/172): layout_type x city x power_backup
Вычислены статистики: mean 
Preparing feature (110/172): layout_type x city x washing_machine
Вычислены статистики: mean 
Preparing feature (111/172): layout_type x city x air_conditioner
Вычислены статистики: mean 
Preparing feature (112/172): layout_type x parking_spaces x power_backup
Вычислены статистики: mean 
Preparing feature (113/172): layout_type x floor x pet_friendly
Вычислены статистики: mean 
Preparing feature (114/172): layout_type x floor x power_backup
Вычислены статистики: mean 
Preparing feature (115/172): layout_type x floor x air_conditioner
Вычислены статистики: mean 
Preparing feature (116/172): layout_type x pet_friendly x lift
Вычислены статистики: mean 
Preparing feature (117/172): layout_type x power_backup x washing_machine
Вычислены статистики: mean 
Preparing feature (118/172): layout_type x power_backup x air_conditioner
Вычислены статистики: mean 
Preparing feature 

-27697.200716309893

RMSE составляет 27697.20 это ниже, чем при использовании статистик квантилей, максимума и минимума. 

Тоже самое наблюдалось при использовании модели случайного леса.

Из этого следует, что категориальные признаки можно кодировать, используя среднее. 

Лучшей оказалась модель случайного леса со схемой кодирования категорий (mean, q25, q50, q75, min, max ). Её результаты на валидационной выборке составили 27602.68 